# Program 5 : Bayes Theorem

## import csv, random and math

In [76]:
import csv
import random
import math

## unnecessary load function which could have been within the Main

In [77]:
def loadcsv(filename):
    reader=csv.reader(open(filename,"r"))
    dataset=list(reader)
    for i in range(len(dataset)):
        dataset[i]=[float(x) for x in dataset[i]]

    return dataset

## function to split data into train and test set

In [78]:
def splitdataset(dataset,splitratio):
    trainsize=int(len(dataset)*splitratio)
    trainset=[]
    copy=list(dataset)
    while(len(trainset)<trainsize):
        index=random.randrange(len(copy))
        trainset.append(copy.pop(index))
    return [trainset,copy]

In [79]:
def separatebyclass(dataset):
    separated={}
    for i in range(len(dataset)):
        vector=dataset[i]
        if(vector[-1] not in separated):
            separated[vector[-1]]=[]
        separated[vector[-1]].append(vector)
    return separated

## Mean function

In [80]:
def mean(numbers):
    return  sum(numbers)/float(len(numbers))

## Standard Deviation

In [81]:
def stdev(numbers):
    avg=mean(numbers)
    variance=sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)

In [82]:
def summarize(dataset):
    summaries=[(mean(attribute),stdev(attribute)) for attribute in zip(*dataset)]
    del summaries[-1]
    return summaries

In [83]:
def summarizebyclass(dataset):
    separated=separatebyclass(dataset)
    summaries={}
    for classvalue,instances in separated.items():
        summaries[classvalue]=summarize(instances)
    return summaries

## to calculate posterior probability

In [84]:
def calculateprobability(x,mean,stdev):
    exponent=math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1/(math.sqrt(2*math.pi)*stdev))*exponent

In [85]:
def calculateclassprobabilities(summaries,inputvector):
    probabilities={}
    for classvalue,classsummaries in summaries.items():
        probabilities[classvalue]=1
        for i in range(len(classsummaries)):
            mean,stdev=classsummaries[i]
            x=inputvector[i]
            probabilities[classvalue]*=calculateprobability(x,mean,stdev)
    return probabilities

In [86]:
def predict(summaries,inputvector):
    probabilities=calculateclassprobabilities(summaries,inputvector)
    bestlabel,bestprob=None,-1
    for classvalue,probability in probabilities.items():
        if bestlabel is None or probability>bestprob:
            bestprob=probability
            bestlabel=classvalue
    return bestlabel

In [87]:
def getpredictions(summaries,testset):
    predictions=[]
    for i in range(len(testset)):
        result=predict(summaries,testset[i])
        predictions.append(result)
    return predictions

In [88]:
def getaccuracy(testset,predictions):
    correct=0
    for i in range(len(testset)):
        if testset[i][-1]==predictions[i]:
            correct+=1
    return (correct/float(len(testset)))*100.0



## main function

In [89]:
def main():
    filename='5.csv'
    splitratio=0.67
    dataset=loadcsv(filename)
    trainingset,testset=splitdataset(dataset,splitratio)
    print('Split {0} rows into Train = {1} and Test {2} rows'.format(len(dataset),len(trainingset),len(testset)))
    summaries=summarizebyclass(trainingset)
    predictions=getpredictions(summaries,testset)
    accuracy=getaccuracy(testset,predictions)
    print('Accuracy of the classifier is: {0}%'.format(accuracy))
    
main()

Split 768 rows into Train = 514 and Test 254 rows
Accuracy of the classifier is: 74.80314960629921%
